In [2]:
import pandas as pd
import numpy as np
df=pd.read_csv('C:\\Users\zhe\\Documents\\python_exercise_code\\basketball_data\\games_season.csv')
from numpy import unique
import tensorflow as tf
# Imports
from keras.layers import Input, Embedding, Flatten,Subtract, BatchNormalization, Concatenate,Dense
from keras.models import Model

### build functional API with multiple inputs and shared layer/model

In [3]:
# Count the unique number of teams
games_season=df
n_teams = (games_season['team_1']).nunique()

#### Embedding: This layer can only be used as the first layer in a model.
they are extremly useful for dealing with high cardinality categorical data
to map the integer team IDs to a decimal rating

In [ ]:
# Create an embedding layer
team_lookup = Embedding(input_dim=n_teams,output_dim=1,input_length=1,name='Team-Strength')
# map from single number, ranging from 1 to 10887, which represent each team's unique ID
# as you are representing each team as a single integer, use an input length of 1
# you want to produce a single team strength rating, use an output dimension of 1
# embedding layer increase dimension, input has two dimensions(rows and columns), 
# but embedding layer add a third dimension, we use flatten
# flatten layer can transform data from multiple dimensions back to two dimensions


#### Model

<img src='https://pbs.twimg.com/media/FFnkyDYXEAEutvO?format=jpg&name=4096x4096' width=1000/>

In [4]:
# define input
    # the number of columns in the input is specified using shape parameter 
    # the shape argument expects a tuple
teamid_in_1 = Input(shape=(1,))
teamid_in_2 = Input(shape=(1,))
# Create an embedding layer
team_lookup = Embedding(input_dim=n_teams,output_dim=1,input_length=1,name='Team-Strength')
    # flatten layer to compact 3-d to 2-d
# shared layers
tensor_1_layer_1=Flatten()(team_lookup(teamid_in_1))
tensor_2_layer_1=Flatten()(team_lookup(teamid_in_2))
# merge layers
score_diff = Subtract()([tensor_1_layer_1, tensor_2_layer_1])
# bulid model by connecting input_tensor and output_tensor
model=Model([teamid_in_1 ,teamid_in_2],score_diff)

home_in = Input(shape=(1,), name='Home-In')
out = Concatenate()([score_diff, home_in])
out = Dense(1)(out)
model=Model([teamid_in_1 ,teamid_in_2,home_in],out)

#### plot model image

In [8]:
import pydot

In [9]:
from keras.utils.vis_utils import plot_model
plot_model(model, to_file ='model.png')
from matplotlib import pyplot as plt 
img = plt.imread('model.png') 
plt.imshow(img)
plt.show()

('You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) ', 'for plot_model/model_to_dot to work.')


FileNotFoundError: [Errno 2] No such file or directory: 'model.png'

#### diff btw concatante and add

In [6]:
import keras
import tensorflow as tf
import keras.backend as K
a = tf.constant([1,2,3])
b = tf.constant([4,5,6])
add = keras.layers.Subtract()
print(K.eval(add([a,b])))
#output: [5 7 9]
concat = keras.layers.Concatenate()
print(K.eval(concat([a,b])))
#output: array([1, 2, 3, 4, 5, 6], dtype=int32)

[-3 -3 -3]
[1 2 3 4 5 6]


#### no embed layer

teamid_in_1 = Input(shape=(1,))
teamid_in_2 = Input(shape=(1,))
Create an embedding layer
score_diff = Subtract()([teamid_in_1, teamid_in_2])
output_tensor= BatchNormalization()(score_diff) 
model=Model([teamid_in_1 ,teamid_in_2],output_tensor)

#### model train

In [7]:
model.compile(optimizer='adam', loss='mean_absolute_error')
# Get the team_1 column from the regular season data
input_1 = games_season['team_1']
# Get the team_2 column from the regular season data
input_2 = games_season['team_2']
home_in= games_season['home']
model.fit([input_1, input_2,home_in],games_season.score_diff,epochs=1,batch_size=2048,validation_split=0.1,
          verbose=True)

138/138 [==============================] - 1s 3ms/step - loss: 12.0497 - val_loss: 12.0556


#### stacking models/shared models

<img src='https://pbs.twimg.com/media/FFnxQq7X0AAt1mr?format=jpg&name=large' width=500/>

In [13]:
games_tourney = pd.read_csv('basketball_data/games_tourney.csv')
in_data_1 = games_tourney['team_1'] 
in_data_2 = games_tourney['team_2'] 
in_data_3 = games_tourney['home']
out_data_1 = games_tourney.score_diff
pred =model.predict([in_data_1, in_data_2, in_data_3])
model.evaluate([in_data_1, in_data_2, in_data_3],out_data_1)

133/133 [==============================] - 0s 892us/step - loss: 11.6844


11.684370994567871

In [4]:
# one input tensor with 3 columns
teamid_in = Input(shape=(3,))

NameError: name 'Input' is not defined

### Two-output models

#### Single model for regression

In [14]:
teamid_in_1 = Input(shape=(1,))
teamid_in_2 = Input(shape=(1,))
# Create an embedding layer
team_lookup = Embedding(input_dim=n_teams,output_dim=1,input_length=1,name='Team-Strength')
tensor_1_layer_1=Flatten()(team_lookup(teamid_in_1))
tensor_2_layer_1=Flatten()(team_lookup(teamid_in_2))
score_diff = Subtract()([tensor_1_layer_1, tensor_2_layer_1])
home_in = Input(shape=(1,), name='Home-In')
out = Concatenate()([score_diff, home_in])
# create output tensor with 2 columns, score_home and score_away
out = Dense(2)(out)
model=Model([teamid_in_1 ,teamid_in_2,home_in],out)

fit model

In [35]:
model.compile(optimizer='adam', loss='mean_absolute_error')
# Get the team_1 column from the regular season data
input_1 = games_season['team_1']
# Get the team_2 column from the regular season data
input_2 = games_season['team_2']
home_in= games_season['home']
model.fit([input_1, input_2,home_in],[games_season.score_1,games_season.score_2],epochs=1,batch_size=2048,validation_split=0.1,
          verbose=True)

138/138 [==============================] - 1s 3ms/step - loss: 71.0238 - val_loss: 64.0984


In [ ]:
input_tensor = Input(shape=(1,))
output_tensor_1 = Dense(1, activation='linear', use_bias=False)(input_tensor)
output_tensor_2 = Dense(1, activation='linear', use_bias=False)(output_tensor_1)
model = Model(input_tensor,[output_tensor_1,output_tensor_2])

model.compile(optimizer='adam',loss ='mean_absolute_error')
model.fit(games_tourney['seed_diff'],
  		  [games_tourney['score_1'],games_tourney['score_2']],
  		  verbose=True,
  		  batch_size=16384,
  		  epochs=1)

#### Single model for classification and regression

In [2]:
from tensorflow.keras.optimizers import Adam

In [3]:
# Create an input layer with 2 columns
input_tensor = Input(shape=(1,))
# Create the first regression output for score_difference
output_tensor_1 = Dense(1, activation='linear', use_bias=False)(input_tensor)
# Create the second classification output for win_or_loss (use the first output as input here)
output_tensor_2 = Dense(1, activation='sigmoid', use_bias=False)(output_tensor_1)
# Create a model with 2 outputs
model = Model(input_tensor, [output_tensor_1, output_tensor_2])
# Import the Adam optimizer

# Compile the model with 2 losses and the Adam optimzer with a higher learning rate
model.compile(loss=['mean_absolute_error', 'binary_crossentropy'], optimizer=Adam(0.01))
# Fit the model to the tournament training data, with 2 inputs and 2 outputs
model.fit(games_tourney[['seed_diff']],
          [games_tourney[['score_diff']], games_tourney[['won']]],
          epochs=10,
          verbose=True,
          batch_size=16384)

NameError: name 'Input' is not defined

In [1]:
# Imports
import matplotlib.pyplot as plt
from keras.utils.vis_utils import plot_model

# Plot the model
plot_model(model, to_file='semi_join.png')

NameError: name 'model' is not defined

In [ ]:
# Display the image
data = plt.imread('semi_join.png')
plt.imshow(data)
plt.show()